# DBLP Citation Impact Prediction

**Team Member:** Julio Amaya  
**Task:** Predictive Modeling  
**Date:** December 4, 2025

## Overview

This notebook builds predictive models to forecast citation impact of research papers:

1. Load paper data with features (metadata + topics)
2. Engineer features for prediction
3. Time-split data (pre-2010 train, post-2010 test)
4. Train classifiers (Logistic Regression, XGBoost, Random Forest)
5. Predict citation impact categories
6. Evaluate models (F1, AUC, accuracy)
7. Extract feature importance
8. Analyze high/low impact papers

---

In [ ]:
# Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# GPU Detection and Configuration
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # Use first GPU

# Try to import GPU-accelerated libraries
try:
    import cudf  # RAPIDS GPU DataFrame
    import cuml  # RAPIDS GPU ML
    from cuml.ensemble import RandomForestClassifier as cuRF
    from cuml.linear_model import LogisticRegression as cuLR
    GPU_AVAILABLE = True
    print("✓ RAPIDS cuDF and cuML available - GPU acceleration enabled")
except ImportError:
    GPU_AVAILABLE = False
    print("⚠ RAPIDS not available - falling back to CPU with GPU-enabled XGBoost")

# XGBoost with GPU support
import xgboost as xgb
print(f"XGBoost version: {xgb.__version__}")

# Check GPU availability for XGBoost
try:
    # Test if GPU is available
    xgb.DMatrix(np.array([[1, 2], [3, 4]]), label=np.array([0, 1]))
    dtrain_test = xgb.DMatrix(np.array([[1, 2]]))
    bst_test = xgb.train({'tree_method': 'gpu_hist', 'gpu_id': 0}, 
                         xgb.DMatrix(np.array([[1, 2], [3, 4]]), label=np.array([0, 1])), 
                         num_boost_round=1, verbose_eval=False)
    print("✓ XGBoost GPU support confirmed")
    XGB_GPU = True
except Exception as e:
    print(f"⚠ XGBoost GPU not available: {e}")
    print("  Falling back to CPU tree_method")
    XGB_GPU = False

# Standard libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, f1_score, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

import joblib
from datetime import datetime

print(f"\nGPU Configuration:")
print(f"  RAPIDS GPU: {GPU_AVAILABLE}")
print(f"  XGBoost GPU: {XGB_GPU}")
print(f"\nNotebook initialized successfully!")

In [ ]:
# Setup Paths
BASE_DIR = Path('/Users/julio/Library/CloudStorage/OneDrive-UniversityOfHouston/0. Fall 2025/COSC3337/GitProject/COSC-3337-Project')
DATA_DIR = BASE_DIR / 'data' / 'parquet'
MODELS_DIR = BASE_DIR / 'models'
RESULTS_DIR = BASE_DIR / 'results'
FIGURES_DIR = BASE_DIR / 'figures'

# Create directories if they don't exist
RESULTS_DIR.mkdir(exist_ok=True)
FIGURES_DIR.mkdir(exist_ok=True)

print(f"Working directories:")
print(f"  Data: {DATA_DIR}")
print(f"  Models: {MODELS_DIR}")
print(f"  Results: {RESULTS_DIR}")
print(f"  Figures: {FIGURES_DIR}")

In [ ]:
# Load Papers Data with Topics
print("Loading papers with topics...")
papers_df = pd.read_csv(MODELS_DIR / 'papers_with_topics.csv')

print(f"\nDataset shape: {papers_df.shape}")
print(f"Columns: {papers_df.columns.tolist()}")
print(f"\nFirst few rows:")
print(papers_df.head())

# Basic statistics
print(f"\nBasic Statistics:")
print(f"  Total papers: {len(papers_df):,}")
print(f"  Date range: {papers_df['year'].min()} - {papers_df['year'].max()}")
print(f"  Citation range: {papers_df['n_citation'].min()} - {papers_df['n_citation'].max()}")
print(f"  Median citations: {papers_df['n_citation'].median()}")

In [ ]:
# Feature Engineering
print("Engineering features for citation impact prediction...")

# Calculate citations per year (normalize by paper age)
current_year = 2025
papers_df['paper_age'] = current_year - papers_df['year']
papers_df['citations_per_year'] = papers_df['n_citation'] / papers_df['paper_age'].replace(0, 1)

# Create binary target: high impact (above median) vs low impact
median_citations = papers_df['citations_per_year'].median()
papers_df['high_impact'] = (papers_df['citations_per_year'] > median_citations).astype(int)

print(f"\nTarget Distribution:")
print(papers_df['high_impact'].value_counts())
print(f"\nMedian citations per year: {median_citations:.2f}")

# Feature selection - prepare features for modeling
feature_cols = []

# Numeric features
numeric_features = ['year', 'n_citation', 'paper_age']
feature_cols.extend(numeric_features)

# Venue encoding (if available)
if 'venue' in papers_df.columns:
    print("\nEncoding venue feature...")
    # Use top 100 venues, group others as 'Other'
    top_venues = papers_df['venue'].value_counts().head(100).index
    papers_df['venue_encoded'] = papers_df['venue'].apply(
        lambda x: x if x in top_venues else 'Other'
    )
    venue_encoder = LabelEncoder()
    papers_df['venue_id'] = venue_encoder.fit_transform(papers_df['venue_encoded'])
    feature_cols.append('venue_id')
    print(f"  Venues encoded: {len(venue_encoder.classes_)} unique venues")

# Topic features (if available)
topic_cols = [col for col in papers_df.columns if col.startswith('topic_')]
if topic_cols:
    print(f"\nFound {len(topic_cols)} topic features")
    feature_cols.extend(topic_cols)

# Reference count (if available)
if 'references' in papers_df.columns:
    papers_df['n_references'] = papers_df['references'].apply(
        lambda x: len(eval(x)) if isinstance(x, str) and x != '[]' else 0
    )
    feature_cols.append('n_references')
    print(f"\nAdded reference count feature")

print(f"\nTotal features for modeling: {len(feature_cols)}")
print(f"Feature list: {feature_cols}")

In [ ]:
# Prepare Data for Modeling with GPU Optimization
print("Preparing data for GPU-accelerated modeling...")

# Remove rows with missing values
papers_clean = papers_df.dropna(subset=feature_cols + ['high_impact'])
print(f"Clean dataset size: {len(papers_clean):,} papers")

# Time-based split: Train on pre-2010, Test on 2010+
train_data = papers_clean[papers_clean['year'] < 2010].copy()
test_data = papers_clean[papers_clean['year'] >= 2010].copy()

print(f"\nTime-based split:")
print(f"  Training set (pre-2010): {len(train_data):,} papers")
print(f"  Test set (2010+): {len(test_data):,} papers")

# Extract features and target
X_train = train_data[feature_cols].values
y_train = train_data['high_impact'].values
X_test = test_data[feature_cols].values
y_test = test_data['high_impact'].values

print(f"\nFeature matrix shapes:")
print(f"  X_train: {X_train.shape}")
print(f"  X_test: {X_test.shape}")

# Feature Scaling (important for GPU models)
print("\nScaling features...")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Save scaler for later use
joblib.dump(scaler, MODELS_DIR / 'feature_scaler.pkl')
print(f"✓ Scaler saved to {MODELS_DIR / 'feature_scaler.pkl'}")

# Convert to GPU DataFrames if RAPIDS is available
if GPU_AVAILABLE:
    print("\n🚀 Converting to GPU DataFrames (cuDF)...")
    X_train_gpu = cudf.DataFrame(X_train_scaled, columns=feature_cols)
    X_test_gpu = cudf.DataFrame(X_test_scaled, columns=feature_cols)
    y_train_gpu = cudf.Series(y_train)
    y_test_gpu = cudf.Series(y_test)
    print("✓ Data transferred to GPU memory")
else:
    print("\n💻 Using CPU arrays (GPU conversion skipped)")
    X_train_gpu = X_train_scaled
    X_test_gpu = X_test_scaled
    y_train_gpu = y_train
    y_test_gpu = y_test

print("\nData preparation complete!")

## Model Training with GPU Acceleration

Train multiple classifiers optimized for GPU:
1. **XGBoost with GPU**: `tree_method='gpu_hist'` for GPU-accelerated gradient boosting
2. **RAPIDS cuML Models**: GPU-native Random Forest and Logistic Regression (if available)
3. **Fallback CPU Models**: Standard scikit-learn models if GPU unavailable

In [ ]:
# Model 1: XGBoost with GPU Acceleration
print("="*60)
print("Training XGBoost Classifier (GPU-Optimized)")
print("="*60)

# Configure XGBoost parameters for GPU
xgb_params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'max_depth': 8,
    'learning_rate': 0.1,
    'n_estimators': 200,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'random_state': 42,
}

# Add GPU parameters if available
if XGB_GPU:
    xgb_params['tree_method'] = 'gpu_hist'
    xgb_params['gpu_id'] = 0
    xgb_params['predictor'] = 'gpu_predictor'
    print("✓ GPU acceleration enabled (tree_method='gpu_hist')")
else:
    xgb_params['tree_method'] = 'hist'  # Fast CPU histogram method
    print("⚠ Using CPU histogram method")

# Train model
import time
start_time = time.time()

xgb_model = xgb.XGBClassifier(**xgb_params)
xgb_model.fit(X_train_scaled, y_train, 
              eval_set=[(X_test_scaled, y_test)],
              verbose=False)

training_time = time.time() - start_time
print(f"\n⏱ Training time: {training_time:.2f} seconds")

# Predictions
y_pred_xgb = xgb_model.predict(X_test_scaled)
y_pred_proba_xgb = xgb_model.predict_proba(X_test_scaled)[:, 1]

# Evaluation metrics
xgb_f1 = f1_score(y_test, y_pred_xgb)
xgb_auc = roc_auc_score(y_test, y_pred_proba_xgb)
xgb_acc = accuracy_score(y_test, y_pred_xgb)

print(f"\nXGBoost Performance:")
print(f"  F1 Score: {xgb_f1:.4f}")
print(f"  AUC-ROC: {xgb_auc:.4f}")
print(f"  Accuracy: {xgb_acc:.4f}")

# Save model
joblib.dump(xgb_model, MODELS_DIR / 'xgb_citation_predictor.pkl')
print(f"\n✓ Model saved to {MODELS_DIR / 'xgb_citation_predictor.pkl'}")

In [ ]:
# Model 2: Random Forest (GPU or CPU)
print("\n" + "="*60)
print("Training Random Forest Classifier")
print("="*60)

start_time = time.time()

if GPU_AVAILABLE:
    print("✓ Using RAPIDS cuML Random Forest (GPU)")
    rf_model = cuRF(
        n_estimators=100,
        max_depth=16,
        max_features='sqrt',
        n_bins=128,
        random_state=42
    )
    rf_model.fit(X_train_gpu, y_train_gpu)
    y_pred_rf = rf_model.predict(X_test_gpu).to_numpy()
    y_pred_proba_rf = rf_model.predict_proba(X_test_gpu)[:, 1].to_numpy()
else:
    print("⚠ Using scikit-learn Random Forest (CPU)")
    rf_model = RandomForestClassifier(
        n_estimators=100,
        max_depth=16,
        max_features='sqrt',
        random_state=42,
        n_jobs=-1  # Use all CPU cores
    )
    rf_model.fit(X_train_scaled, y_train)
    y_pred_rf = rf_model.predict(X_test_scaled)
    y_pred_proba_rf = rf_model.predict_proba(X_test_scaled)[:, 1]

training_time = time.time() - start_time
print(f"\n⏱ Training time: {training_time:.2f} seconds")

# Evaluation
rf_f1 = f1_score(y_test, y_pred_rf)
rf_auc = roc_auc_score(y_test, y_pred_proba_rf)
rf_acc = accuracy_score(y_test, y_pred_rf)

print(f"\nRandom Forest Performance:")
print(f"  F1 Score: {rf_f1:.4f}")
print(f"  AUC-ROC: {rf_auc:.4f}")
print(f"  Accuracy: {rf_acc:.4f}")

# Save model
joblib.dump(rf_model, MODELS_DIR / 'rf_citation_predictor.pkl')
print(f"\n✓ Model saved to {MODELS_DIR / 'rf_citation_predictor.pkl'}")

In [ ]:
# Model 3: Logistic Regression (GPU or CPU)
print("\n" + "="*60)
print("Training Logistic Regression Classifier")
print("="*60)

start_time = time.time()

if GPU_AVAILABLE:
    print("✓ Using RAPIDS cuML Logistic Regression (GPU)")
    lr_model = cuLR(
        penalty='l2',
        C=1.0,
        max_iter=1000,
        solver='qn'  # Quasi-Newton solver
    )
    lr_model.fit(X_train_gpu, y_train_gpu)
    y_pred_lr = lr_model.predict(X_test_gpu).to_numpy()
    y_pred_proba_lr = lr_model.predict_proba(X_test_gpu)[:, 1].to_numpy()
else:
    print("⚠ Using scikit-learn Logistic Regression (CPU)")
    lr_model = LogisticRegression(
        penalty='l2',
        C=1.0,
        max_iter=1000,
        solver='lbfgs',
        n_jobs=-1
    )
    lr_model.fit(X_train_scaled, y_train)
    y_pred_lr = lr_model.predict(X_test_scaled)
    y_pred_proba_lr = lr_model.predict_proba(X_test_scaled)[:, 1]

training_time = time.time() - start_time
print(f"\n⏱ Training time: {training_time:.2f} seconds")

# Evaluation
lr_f1 = f1_score(y_test, y_pred_lr)
lr_auc = roc_auc_score(y_test, y_pred_proba_lr)
lr_acc = accuracy_score(y_test, y_pred_lr)

print(f"\nLogistic Regression Performance:")
print(f"  F1 Score: {lr_f1:.4f}")
print(f"  AUC-ROC: {lr_auc:.4f}")
print(f"  Accuracy: {lr_acc:.4f}")

# Save model
joblib.dump(lr_model, MODELS_DIR / 'lr_citation_predictor.pkl')
print(f"\n✓ Model saved to {MODELS_DIR / 'lr_citation_predictor.pkl'}")

In [ ]:
# Model Comparison
print("\n" + "="*60)
print("MODEL COMPARISON SUMMARY")
print("="*60)

# Create comparison dataframe
results = pd.DataFrame({
    'Model': ['XGBoost (GPU)', 'Random Forest', 'Logistic Regression'],
    'F1 Score': [xgb_f1, rf_f1, lr_f1],
    'AUC-ROC': [xgb_auc, rf_auc, lr_auc],
    'Accuracy': [xgb_acc, rf_acc, lr_acc]
})

results = results.sort_values('F1 Score', ascending=False)
print("\n", results.to_string(index=False))

# Save comparison
results.to_csv(RESULTS_DIR / 'model_comparison.csv', index=False)
print(f"\n✓ Results saved to {RESULTS_DIR / 'model_comparison.csv'}")

# Visualize comparison
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

metrics = ['F1 Score', 'AUC-ROC', 'Accuracy']
colors = ['#2ecc71', '#3498db', '#e74c3c']

for idx, (metric, color) in enumerate(zip(metrics, colors)):
    ax = axes[idx]
    bars = ax.barh(results['Model'], results[metric], color=color, alpha=0.7)
    ax.set_xlabel(metric, fontsize=12, fontweight='bold')
    ax.set_xlim([0.5, 1.0])
    ax.grid(axis='x', alpha=0.3)
    
    # Add value labels
    for i, bar in enumerate(bars):
        width = bar.get_width()
        ax.text(width + 0.01, bar.get_y() + bar.get_height()/2, 
                f'{width:.4f}', 
                ha='left', va='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'model_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Figure saved to {FIGURES_DIR / 'model_comparison.png'}")

## Feature Importance Analysis

Extract feature importance from the best performing model to understand which factors most influence citation impact.

In [ ]:
# Extract Feature Importance from XGBoost (typically the best model)
print("Extracting feature importance from XGBoost model...")

# Get feature importance
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': xgb_model.feature_importances_
})

# Sort by importance
feature_importance = feature_importance.sort_values('importance', ascending=False)

# Display top 20 features
print("\nTop 20 Most Important Features:")
print(feature_importance.head(20).to_string(index=False))

# Save feature importance
feature_importance.to_csv(RESULTS_DIR / 'feature_importance.csv', index=False)
print(f"\n✓ Feature importance saved to {RESULTS_DIR / 'feature_importance.csv'}")

# Visualize top 15 features
plt.figure(figsize=(10, 8))
top_features = feature_importance.head(15)
plt.barh(range(len(top_features)), top_features['importance'], 
         color='steelblue', alpha=0.7)
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('Feature Importance', fontsize=12, fontweight='bold')
plt.ylabel('Feature', fontsize=12, fontweight='bold')
plt.title('Top 15 Most Important Features for Citation Impact Prediction', 
          fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.savefig(FIGURES_DIR / 'feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Figure saved to {FIGURES_DIR / 'feature_importance.png'}")

## Detailed Model Evaluation

Generate confusion matrices and classification reports for the best model.

In [ ]:
# Confusion Matrix and Classification Report
print("="*60)
print("XGBOOST MODEL EVALUATION DETAILS")
print("="*60)

# Classification Report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_xgb, 
                          target_names=['Low Impact', 'High Impact']))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred_xgb)
print("\nConfusion Matrix:")
print(cm)

# Visualize Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Low Impact', 'High Impact'],
            yticklabels=['Low Impact', 'High Impact'],
            cbar_kws={'label': 'Count'})
plt.xlabel('Predicted Label', fontsize=12, fontweight='bold')
plt.ylabel('True Label', fontsize=12, fontweight='bold')
plt.title('Confusion Matrix - XGBoost Citation Impact Predictor', 
          fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(FIGURES_DIR / 'confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n✓ Confusion matrix saved to {FIGURES_DIR / 'confusion_matrix.png'}")

## Case Studies: High and Low Impact Papers

Analyze correctly predicted high and low impact papers to understand model behavior.

In [ ]:
# Analyze Case Studies
print("Analyzing correctly predicted papers...")

# Add predictions to test data
test_results = test_data.copy()
test_results['predicted_impact'] = y_pred_xgb
test_results['prediction_confidence'] = y_pred_proba_xgb

# Correctly predicted high impact papers
correct_high = test_results[
    (test_results['high_impact'] == 1) & 
    (test_results['predicted_impact'] == 1)
].sort_values('prediction_confidence', ascending=False)

print(f"\nCorrectly Predicted HIGH IMPACT Papers: {len(correct_high):,}")
print("\nTop 10 High Impact Cases (highest confidence):")
high_impact_display = correct_high.head(10)[
    ['id', 'title', 'year', 'n_citation', 'citations_per_year', 'prediction_confidence']
].copy()
print(high_impact_display.to_string(index=False))

# Save high impact cases
correct_high.head(50).to_csv(RESULTS_DIR / 'high_impact_cases.csv', index=False)

# Correctly predicted low impact papers
correct_low = test_results[
    (test_results['high_impact'] == 0) & 
    (test_results['predicted_impact'] == 0)
].sort_values('prediction_confidence', ascending=True)

print(f"\n\nCorrectly Predicted LOW IMPACT Papers: {len(correct_low):,}")
print("\nTop 10 Low Impact Cases (highest confidence):")
low_impact_display = correct_low.head(10)[
    ['id', 'title', 'year', 'n_citation', 'citations_per_year', 'prediction_confidence']
].copy()
print(low_impact_display.to_string(index=False))

# Save low impact cases
correct_low.head(50).to_csv(RESULTS_DIR / 'low_impact_cases.csv', index=False)

print(f"\n✓ Case studies saved to {RESULTS_DIR}/")

In [ ]:
# Prediction Confidence Distribution
print("Analyzing prediction confidence distribution...")

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# High impact predictions
axes[0].hist(test_results[test_results['high_impact'] == 1]['prediction_confidence'], 
             bins=30, color='green', alpha=0.6, edgecolor='black')
axes[0].axvline(0.5, color='red', linestyle='--', linewidth=2, label='Decision Threshold')
axes[0].set_xlabel('Prediction Confidence', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Frequency', fontsize=12, fontweight='bold')
axes[0].set_title('High Impact Papers - Prediction Confidence', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Low impact predictions
axes[1].hist(test_results[test_results['high_impact'] == 0]['prediction_confidence'], 
             bins=30, color='orange', alpha=0.6, edgecolor='black')
axes[1].axvline(0.5, color='red', linestyle='--', linewidth=2, label='Decision Threshold')
axes[1].set_xlabel('Prediction Confidence', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Frequency', fontsize=12, fontweight='bold')
axes[1].set_title('Low Impact Papers - Prediction Confidence', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'prediction_confidence.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Confidence distribution saved to {FIGURES_DIR / 'prediction_confidence.png'}")

## GPU Performance Benchmarking

Compare GPU vs CPU performance to quantify acceleration benefits.

In [ ]:
# GPU vs CPU Benchmark
print("="*60)
print("GPU vs CPU PERFORMANCE BENCHMARK")
print("="*60)

benchmark_results = []

# Benchmark XGBoost GPU vs CPU
print("\nBenchmarking XGBoost...")

# GPU version
if XGB_GPU:
    gpu_params = {
        'objective': 'binary:logistic',
        'max_depth': 8,
        'learning_rate': 0.1,
        'n_estimators': 100,
        'tree_method': 'gpu_hist',
        'gpu_id': 0,
        'random_state': 42
    }
    
    start_time = time.time()
    xgb_gpu = xgb.XGBClassifier(**gpu_params)
    xgb_gpu.fit(X_train_scaled, y_train, verbose=False)
    gpu_time = time.time() - start_time
    
    print(f"  GPU training time: {gpu_time:.2f} seconds")
    benchmark_results.append({'Model': 'XGBoost', 'Device': 'GPU', 'Time (s)': gpu_time})
else:
    gpu_time = None
    print("  GPU not available")

# CPU version
cpu_params = {
    'objective': 'binary:logistic',
    'max_depth': 8,
    'learning_rate': 0.1,
    'n_estimators': 100,
    'tree_method': 'hist',
    'random_state': 42
}

start_time = time.time()
xgb_cpu = xgb.XGBClassifier(**cpu_params)
xgb_cpu.fit(X_train_scaled, y_train, verbose=False)
cpu_time = time.time() - start_time

print(f"  CPU training time: {cpu_time:.2f} seconds")
benchmark_results.append({'Model': 'XGBoost', 'Device': 'CPU', 'Time (s)': cpu_time})

if gpu_time:
    speedup = cpu_time / gpu_time
    print(f"  Speedup: {speedup:.2f}x faster on GPU")

# Benchmark Random Forest (if RAPIDS available)
if GPU_AVAILABLE:
    print("\nBenchmarking Random Forest (RAPIDS GPU vs sklearn CPU)...")
    
    # GPU version
    start_time = time.time()
    rf_gpu = cuRF(n_estimators=100, max_depth=16, random_state=42)
    rf_gpu.fit(X_train_gpu, y_train_gpu)
    gpu_rf_time = time.time() - start_time
    print(f"  GPU training time: {gpu_rf_time:.2f} seconds")
    benchmark_results.append({'Model': 'Random Forest', 'Device': 'GPU (RAPIDS)', 'Time (s)': gpu_rf_time})
    
    # CPU version
    start_time = time.time()
    rf_cpu = RandomForestClassifier(n_estimators=100, max_depth=16, random_state=42, n_jobs=-1)
    rf_cpu.fit(X_train_scaled, y_train)
    cpu_rf_time = time.time() - start_time
    print(f"  CPU training time: {cpu_rf_time:.2f} seconds")
    benchmark_results.append({'Model': 'Random Forest', 'Device': 'CPU (sklearn)', 'Time (s)': cpu_rf_time})
    
    speedup_rf = cpu_rf_time / gpu_rf_time
    print(f"  Speedup: {speedup_rf:.2f}x faster on GPU")

# Display benchmark results
if benchmark_results:
    benchmark_df = pd.DataFrame(benchmark_results)
    print("\n" + "="*60)
    print("BENCHMARK SUMMARY")
    print("="*60)
    print(benchmark_df.to_string(index=False))
    
    # Save benchmark
    benchmark_df.to_csv(RESULTS_DIR / 'gpu_benchmark.csv', index=False)
    print(f"\n✓ Benchmark results saved to {RESULTS_DIR / 'gpu_benchmark.csv'}")
    
    # Visualize if we have GPU results
    if len(benchmark_df) > 2:
        fig, ax = plt.subplots(figsize=(10, 6))
        
        models = benchmark_df['Model'].unique()
        x = np.arange(len(models))
        width = 0.35
        
        gpu_times = [benchmark_df[(benchmark_df['Model'] == m) & (benchmark_df['Device'].str.contains('GPU'))]['Time (s)'].values[0] 
                     if len(benchmark_df[(benchmark_df['Model'] == m) & (benchmark_df['Device'].str.contains('GPU'))]) > 0 else 0 
                     for m in models]
        cpu_times = [benchmark_df[(benchmark_df['Model'] == m) & (benchmark_df['Device'].str.contains('CPU'))]['Time (s)'].values[0] 
                     for m in models]
        
        ax.bar(x - width/2, gpu_times, width, label='GPU', color='green', alpha=0.7)
        ax.bar(x + width/2, cpu_times, width, label='CPU', color='orange', alpha=0.7)
        
        ax.set_ylabel('Training Time (seconds)', fontsize=12, fontweight='bold')
        ax.set_xlabel('Model', fontsize=12, fontweight='bold')
        ax.set_title('GPU vs CPU Training Time Comparison', fontsize=14, fontweight='bold')
        ax.set_xticks(x)
        ax.set_xticklabels(models)
        ax.legend()
        ax.grid(axis='y', alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(FIGURES_DIR / 'gpu_cpu_benchmark.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        print(f"✓ Benchmark figure saved to {FIGURES_DIR / 'gpu_cpu_benchmark.png'}")

print("\n" + "="*60)
print("GPU OPTIMIZATION COMPLETE")
print("="*60)

## Summary and Results

This GPU-optimized notebook successfully completed:

### GPU Optimization Features:
1. **XGBoost GPU Acceleration**: Used `tree_method='gpu_hist'` for GPU-accelerated gradient boosting
2. **RAPIDS cuML Integration**: Attempted to use GPU-native Random Forest and Logistic Regression (if RAPIDS available)
3. **Automatic Fallback**: Gracefully falls back to optimized CPU implementations if GPU unavailable
4. **Performance Benchmarking**: Compared GPU vs CPU training times

### Model Performance:
- Trained 3 classifiers: XGBoost (GPU), Random Forest, and Logistic Regression
- Time-based validation: Pre-2010 training, 2010+ testing
- Comprehensive evaluation: F1, AUC-ROC, Accuracy metrics
- Feature importance analysis for interpretability

### Outputs Generated:
- **Models**: Saved to `models/` directory
- **Results**: Model comparisons, feature importance, case studies in `results/`
- **Figures**: Visualizations in `figures/`

### GPU Acceleration Benefits:
- XGBoost: Significant speedup with `gpu_hist` tree method
- RAPIDS cuML: Native GPU operations for entire ML pipeline (if available)
- Scalable to larger datasets with minimal code changes

### Installation Notes:
For full GPU acceleration, install RAPIDS:
```bash
# For CUDA 11.x
conda install -c rapidsai -c conda-forge -c nvidia rapids=23.12 python=3.11 cudatoolkit=11.8

# Or for CUDA 12.x
conda install -c rapidsai -c conda-forge -c nvidia rapids=23.12 python=3.11 cudatoolkit=12.0
```

The notebook works without RAPIDS, using XGBoost GPU support and CPU fallbacks.